##Encriptar imagen con RSA

### Generar clave

In [ ]:
import random
import math

def generar_dos_primos_altos():
    primos = []
    count = 0

    while count < 2:
        num = random.randint(1, 300)

        if es_primo(num):
            primos.append(num)
            count += 1
    p_a = primos[0]
    p_b = primos[1]
    return p_a,p_b


def es_primo(num, k=5):
    if num < 2:
        return False
    if num in (2, 3):
        return True
    if num % 2 == 0:
        return False
    r, d = 0, num - 1
    while d % 2 == 0:
        r += 1
        d //= 2
    for _ in range(k):
        a = random.randint(2, num - 2)
        x = pow(a, d, num)
        if x == 1 or x == num - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, num)
            if x == num - 1:
                break
        else:
            return False
    return True

def generar_claves(): #RSA
    # Generar números primos p y q
    #p = 101
    #q = 311

    p,q = generar_dos_primos_altos()

    # Calcular n y phi(n)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    # Encontrar un número e tal que 1 < e < phi(n) y gcd(e, phi(n)) = 1
    e = random.randint(2, phi_n - 1)
    while math.gcd(e, phi_n) != 1:
        e = random.randint(2, phi_n - 1)

    # Encontrar el inverso multiplicativo de e modulo phi(n) usando el algoritmo extendido de Euclides
    d = pow(e, -1, phi_n)

    # Devolver las claves pública y privada
    clave_publica = (e, n)
    clave_privada = (d, n)
    return clave_publica, clave_privada

### Encriptar y desencriptar

In [ ]:
    # Generar claves
    clave_publica, clave_privada = generar_claves()
    print(clave_publica, clave_privada)

(143, 346) (83, 346)


In [ ]:
import numpy as np
import cv2
from PIL import Image

def encriptar(numero, clave_publica):
    e, n = clave_publica
    numero_encriptado = pow(numero, e, n)
    return numero_encriptado

def desencriptar(numero_encriptado, clave_privada):
    d, n = clave_privada
    numero_desencriptado = pow(numero_encriptado, d, n)
    return numero_desencriptado

def encriptar_imagen(nombre_imagen,clave):
    img = cv2.imread(nombre_imagen + ".jpg")
    # Inicializar la matriz encriptada
    matriz_encriptada = np.zeros_like(img, dtype=np.uint16)
    matriz_RSA= np.zeros_like(img, dtype=np.uint16)
    # Encriptar los canales de color (RGB) de cada píxel
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                pixel = img[i, j, k]
                pixel_encriptado = encriptar(int(pixel), clave)
                matriz_encriptada[i, j, k] = pixel_encriptado
                matriz_RSA[i, j, k] = pixel_encriptado
    # Guardar la matriz encriptada como una imagen en formato jpg
    imagen_encriptada = Image.fromarray(matriz_encriptada.astype(np.uint8))
    imagen_encriptada.save(nombre_imagen + '_encrip.jpg')
    return matriz_RSA

def desencriptar_imagen(nombre_imagen, clave, matriz):
    img = cv2.imread(nombre_imagen + '_encrip.jpg')
    orden=[2,1,0]
    # Inicializar la matriz desencriptada
    matriz_desencriptada = np.zeros_like(img, dtype=np.uint16)
    # Desencriptar los canales de color (RGB) de cada píxel
    for i in range(matriz.shape[0]):
        for j in range(matriz.shape[1]):
            for k in range(matriz.shape[2]):
                pixel = matriz[i, j, k]
                pixel_desencriptado = desencriptar(int(pixel), clave)
                matriz_desencriptada[i, j, orden[k]] = pixel_desencriptado
    # Guardar la matriz desencriptada como una imagen en formato jpg
    imagen_desencriptada = Image.fromarray(matriz_desencriptada.astype(np.uint8))
    imagen_desencriptada.save(nombre_imagen+'_desencrip.jpg')
    return

def pruebas_encriptación(imagen):
    img = cv2.imread(imagen)
    matriz_RSA = np.zeros_like(img, dtype=np.uint16)
    matriz_RSA = encriptar_imagen(imagen.replace('.jpg', ""),clave_publica)
    desencriptar_imagen(imagen.replace('.jpg', ""),clave_privada, matriz_RSA)
    return

#### Ejemplos de encriptación

In [ ]:
pruebas_encriptación("gato.jpg")

In [ ]:
pruebas_encriptación("qr.jpg")

In [ ]:
pruebas_encriptación("ima002.jpg")